### Lease

In [ ]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV ,lang='english',check=False)

: 

#### Simple Lease

In [11]:
lease = ["Lease"
            ,{"fixRental": 12.0
            ,"originTerm": 24
            ,"freq": ["DayOfMonth",15]
            ,"originDate": "2022-01-05"
            ,"status":"Current"
            ,"remainTerm": 24}]

In [ ]:
r = localAPI.runAsset("2022-01-02"
                     ,[lease]
                     ,poolAssump=("Pool"
                                    ,("Lease", {"Days":30}, {"AnnualIncrease":0.0}, '2023-01-01')
                                    ,None
                                    ,None)
                     ,read=True)

r[0].tail()

: 